# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,15.59,71,80,3.41,CL,1702313195
1,1,chinchilla,-26.7500,150.6333,19.51,80,99,6.29,AU,1702313377
2,2,port lincoln,-34.7333,135.8667,15.93,88,42,1.51,AU,1702313128
3,3,talnakh,69.4865,88.3972,-24.89,97,100,4.47,RU,1702313380
4,4,bredasdorp,-34.5322,20.0403,20.04,55,0,4.74,ZA,1702313381
...,...,...,...,...,...,...,...,...,...,...
553,553,minas de marcona,-15.2119,-75.1103,22.12,57,96,5.77,PE,1702314653
554,554,byelaazyorsk,52.4731,25.1784,0.82,99,100,5.94,BY,1702314654
555,555,crystal city,28.6775,-99.8281,13.06,35,0,3.09,US,1702314655
556,556,carutapera,-1.1950,-46.0200,31.22,58,97,5.29,BR,1702314554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

UsageError: %%capture is a cell magic, but the cell body is empty.


In [4]:
# Configure the map plot
#humidity_cities = city_data_df["Humidity"].astype(int)

# Display the map
humidity_map = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          geo = True,
                                          color = "City",
                                          alpha = 0.6,
                                          size = "Humidity",
                                          frame_width = 800,
                                          frame_height = 600,
                                          tiles = "OSM",
                                          hover_cols = ["City", "Humidity"]
                                          )

humidity_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#Ideal weather
#Max 30, Min 22 
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 22)]

#Wind speed less than 5.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5.5)]

#Cloudiness up to 3 miles/h
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 3)]

#Humidity between 65 and 100
city_data_df = city_data_df.loc[(city_data_df["Humidity"]< 100) & (city_data_df["Humidity"] > 50)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,port elizabeth,-33.9180,25.5701,22.16,76,0,4.12,ZA,1702313264
136,136,carnarvon,-24.8667,113.6333,24.33,62,0,4.47,AU,1702313637
184,184,cabo san lucas,22.8909,-109.9124,27.09,68,0,2.06,MX,1702313631
257,257,talara,-4.5772,-81.2719,28.42,61,0,5.14,PE,1702313939
383,383,salalah,17.0151,54.0924,24.05,57,0,2.57,OM,1702314261
391,391,callao,-12.0667,-77.1500,23.16,68,0,2.57,PE,1702314167
459,459,port hedland,-20.3167,118.5667,23.03,68,0,3.60,AU,1702314455
491,491,pisco,-13.7000,-76.2167,23.34,73,0,4.12,PE,1702314529


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Lat", "Lng", "Country", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Country,Humidity,Hotel Name
33,port elizabeth,-33.9180,25.5701,ZA,76,
136,carnarvon,-24.8667,113.6333,AU,62,
184,cabo san lucas,22.8909,-109.9124,MX,68,
257,talara,-4.5772,-81.2719,PE,61,
383,salalah,17.0151,54.0924,OM,57,
391,callao,-12.0667,-77.1500,PE,68,
459,port hedland,-20.3167,118.5667,AU,68,
491,pisco,-13.7000,-76.2167,PE,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
carnarvon - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
talara - nearest hotel: Hospedaje El Aypate
salalah - nearest hotel: Muscat International Hotel
callao - nearest hotel: Casa Ronald
port hedland - nearest hotel: The Esplanade Hotel
pisco - nearest hotel: La Portada


,City,Lat,Lng,Country,Humidity,Hotel Name
33,port elizabeth,-33.9180,25.5701,ZA,76,Waterford Hotel
136,carnarvon,-24.8667,113.6333,AU,62,No hotel found
184,cabo san lucas,22.8909,-109.9124,MX,68,Comfort Rooms
257,talara,-4.5772,-81.2719,PE,61,Hospedaje El Aypate
383,salalah,17.0151,54.0924,OM,57,Muscat International Hotel
391,callao,-12.0667,-77.1500,PE,68,Casa Ronald
459,port hedland,-20.3167,118.5667,AU,68,The Esplanade Hotel
491,pisco,-13.7000,-76.2167,PE,73,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

UsageError: %%capture is a cell magic, but the cell body is empty.


In [9]:
# Configure the map plot

hotel_country_map = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          geo = True,
                                          color = "City",
                                          alpha = 0.6,
                                          size = "Humidity",
                                          frame_width = 800,
                                          frame_height = 600,
                                          tiles = "OSM",
                                          hover_cols = ["Hotel Name", "Country"]
                                          )
hotel_country_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)